In [13]:
%pip install folium
%pip install geopy
%pip install requests



Note: you may need to restart the kernel to use updated packages.


In [1]:
import folium
import requests
import json
from geopy.distance import great_circle
from geopy.point import Point

In [10]:
# API endpoint
api_endpoint = "https://api.open-meteo.com/v1/forecast"


weather_parameters = [
    "temperature_2m",
    "relativehumidity_2m",
    "precipitation",
    "cloudcover",
    "visibility",
    "evapotranspiration",
    "windspeed_10m",
    "winddirection_10m",
    "soil_temperature_6cm",
    "soil_moisture_0_to_1cm"
]

# Input coordinates for the city of Strasbourg
latitude = 48.584614
longitude = 7.7507127

# We create a reference point
reference_point = Point(latitude, longitude)

# We fix à minimum distance (5 km) from the center of the reference
min_distance = 0.5  # In kilometers

# We create a list to store the coordinates
coordinates = []

# And calculate coordinates for the 4 corner points
corner_points = []

for direction in [0, 90, 180, 270]:
    new_point = great_circle(kilometers=min_distance).destination(reference_point, direction)
    corner_points.append((new_point.latitude, new_point.longitude))

# We calculate coordinates for the additional points on each side
side_points = []

for i in range(0, 5):  # Generate 5 points for each side
    for corner in corner_points:
        start_point = Point(corner[0], corner[1])
        direction = great_circle(kilometers=i * min_distance).destination(start_point, 90)  # East direction
        side_points.append((direction.latitude, direction.longitude))

# Then add the center point
center_latitude = (corner_points[0][0] + corner_points[2][0]) / 2
center_longitude = (corner_points[0][1] + corner_points[2][1]) / 2
center_point = (center_latitude, center_longitude)

# Combine the corner points, side points, and center point
all_points = corner_points + side_points + [center_point]

# Let's display the coordinates of all 25 points
for i, point in enumerate(all_points):
    print(f"Point {i + 1} - Latitude: {point[0]}, Longitude: {point[1]}")

# Then we generate the API request for each point and store the coordinates
for i, (latitude, longitude) in enumerate(all_points):
    # Build the API request URL with the coordinates and selected weather parameters
    api_parameters = ",".join(weather_parameters)
    url = f"{api_endpoint}?latitude={latitude}&longitude={longitude}&hourly={api_parameters}"
    
    # We make the API request
    response = requests.get(url)
    
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        coordinates.append((latitude, longitude, data))
    else:
        print(f"Failed to retrieve temperature data for Point {i + 1}")

# We finally save the data to a JSON file
with open("temperature_data.json", "w") as json_file:
    json.dump(coordinates, json_file, indent=4)
    

print("Temperature data has been saved to temperature_data.json")

    

Point 1 - Latitude: 48.58911060167747, Longitude: 7.750712700000001
Point 2 - Latitude: 48.58461379996759, Longitude: 7.7575101473759505
Point 3 - Latitude: 48.58011739832254, Longitude: 7.750712700000001
Point 4 - Latitude: 48.58461379996759, Longitude: 7.743915252624052
Point 5 - Latitude: 48.58911060167747, Longitude: 7.750712700000001
Point 6 - Latitude: 48.5846137999676, Longitude: 7.7575101473759505
Point 7 - Latitude: 48.58011739832254, Longitude: 7.750712700000001
Point 8 - Latitude: 48.5846137999676, Longitude: 7.743915252624052
Point 9 - Latitude: 48.58911040161341, Longitude: 7.757510752222874
Point 10 - Latitude: 48.58461359993519, Longitude: 7.764307594724996
Point 11 - Latitude: 48.58011719832177, Longitude: 7.757509542678507
Point 12 - Latitude: 48.58461359993519, Longitude: 7.750712699973096
Point 13 - Latitude: 48.58910980142124, Longitude: 7.76430880433809
Point 14 - Latitude: 48.58461299983796, Longitude: 7.771105041966429
Point 15 - Latitude: 48.58011659831948, Long

In [1]:
# We open the file containing the JSON weather data
with open('temperature_data.json', 'r') as f:
    # Load the JSON data from the file
    data = json.load(f)

# Then create a map centered around the center point
center_latitude, center_longitude, _ = data[-1]
m = folium.Map(location=[center_latitude, center_longitude], zoom_start=15)

# And add a tile layer for the map
folium.TileLayer(
    'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'
).add_to(m)

# We add markers for each point and display weather data as popups
for point_data in data:
    latitude, longitude, weather_data = point_data
    tooltip = f"Weather @ {latitude}, {longitude}"
    
    # We construct a popup displaying multiple weather parameters
    popup_content = "<b>Weather Data</b><br>"
    for parameter, values in weather_data['hourly'].items():
        if values:
            value = values[0]
            popup_content += f"{parameter}: {value}<br>"
    
    folium.Marker(
        location=[latitude, longitude],
        tooltip=tooltip,
        popup=popup_content,
        icon=folium.Icon(icon="location-dot")
    ).add_to(m)

# Display the map
m




NameError: name 'json' is not defined